# Рекомендация тарифов

## 1. Изучение данных

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier


data = pd.read_csv('/datasets/users_behavior.csv')
data



,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


Данные открыты, целевой признак - столбец is_ultra.

## 2. Выборки

In [2]:
#delim na viborki
data_valid_train, data_test = train_test_split(data, test_size=0.2, random_state=12345)
data_train, data_valid = train_test_split(data_valid_train, test_size=0.25, random_state=12345)



# priznak i cel
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

Данные разбиты на 3 части в пропорции 6:2:2 (обучающая, валидационная, тестовая).
Заданы признаки и целевые признаки.

## 3. Модели

In [3]:
# подбираем параметры разных моделей
print('дерево решений')
for i in range(1, 7):
    model1 = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model1.fit(features_train, target_train)
    train_predictions1 = model1.predict(features_valid)
    print("max_depth =", i, "accuracy:",accuracy_score(target_valid, train_predictions1))


print()


print('случайный лес')
#for estimators in range(10, 101, 10):
    #model2 = RandomForestClassifier(n_estimators=estimators, max_depth=10, random_state=12345)
    #model2.fit(features_train, target_train)
    #train_predictions2 = model2.predict(features_valid)
    #print("n_estimators =", estimators, ":", accuracy_score(target_valid, train_predictions2))
    
    
for estimators in range(10, 101, 10):
    for depth in range(1,11,1):
        model2 = RandomForestClassifier(n_estimators=estimators, max_depth=depth, random_state=12345)
        model2.fit(features_train, target_train)
        train_predictions2 = model2.predict(features_valid)
        print("n_estimators =", estimators,"depth =", depth, "accuracy:", accuracy_score(target_valid, train_predictions2))
    
    

    
print()  

print('логистическая регрессия')
model3 = LogisticRegression()
model3.fit(features_train, target_train)
train_predictions3 = model3.predict(features_valid)
print('логистическая регрессия', "accuracy:", accuracy_score(target_valid, train_predictions3))

    


дерево решений
max_depth = 1 accuracy: 0.7387247278382582
max_depth = 2 accuracy: 0.7573872472783826
max_depth = 3 accuracy: 0.7651632970451011
max_depth = 4 accuracy: 0.7636080870917574
max_depth = 5 accuracy: 0.7589424572317263
max_depth = 6 accuracy: 0.7573872472783826

случайный лес
n_estimators = 10 depth = 1 accuracy: 0.7309486780715396
n_estimators = 10 depth = 2 accuracy: 0.7636080870917574
n_estimators = 10 depth = 3 accuracy: 0.7713841368584758
n_estimators = 10 depth = 4 accuracy: 0.7776049766718507
n_estimators = 10 depth = 5 accuracy: 0.7776049766718507
n_estimators = 10 depth = 6 accuracy: 0.7853810264385692
n_estimators = 10 depth = 7 accuracy: 0.7869362363919129
n_estimators = 10 depth = 8 accuracy: 0.7853810264385692
n_estimators = 10 depth = 9 accuracy: 0.7947122861586314
n_estimators = 10 depth = 10 accuracy: 0.7900466562986003
n_estimators = 20 depth = 1 accuracy: 0.7325038880248833
n_estimators = 20 depth = 2 accuracy: 0.7620528771384136
n_estimators = 20 depth = 3

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


У дерева решение проверяли разные значения глубины.
У дерева решений макс. accuracy 0,795 при глубине 3, дальше она снижается.

У случайного леса проверяли разные значения количества деревьев. 
У случайного леса accuracy примерно на одном уровне, макс. 0,797 при количестве деревьев 50.
У лог.регресии минимальная accuracy, 0,69.
Тк самая высокая точность у леса - используем его.
Конечная модель - лес с глубиной 10, кол-вом деревьев 50.

## 4. Проверка модели

In [4]:
model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=12345)
model.fit(features_train, target_train)
test_predictions = model.predict(features_test)
print(accuracy_score(target_test, test_predictions))

0.7993779160186625


accuracy итоговой модели на тестовой выборке 0,799 - успешно.

## 5. Проверка на адекватность

In [7]:
#проверка со случайным результатом 
dummy_clf_uniform = DummyClassifier(strategy="uniform")
dummy_clf_uniform.fit(features_train, target_train)
dummy_uni_pred = dummy_clf_uniform.predict(features_test)
print(dummy_clf_uniform.score(target_test,dummy_uni_pred))



0.48055987558320373


Итоговая модель проверена на адекватность.